In [1]:
import sys
import helpers, folder
from attribute_templates import *
from object_templates import *
from utilities import *
import os
import gemd
from gemd.entity.object import MaterialRun, ProcessRun, IngredientRun, MeasurementRun
from gemd.entity import PerformedSource, FileLink
from gemd.entity.attribute import Property, Parameter, Condition, PropertyAndConditions
from gemd.json import GEMDJson
from pathlib import Path

In [2]:
encoder = GEMDJson()

### SPECs

In [3]:
from gemd.entity.object import MaterialRun, ProcessRun, IngredientRun, MeasurementRun

In [4]:
# composition_material_spec = generate_composition(conditions=[], 
#                                                     parameters=[], 
#                                                     output_mat_name='Composition', 
#                                                     output_mat_properties=[])
# alloy_material_spec = fabricate_alloy(
#                 ingredients=[composition_material_spec],
#                 conditions=[], 
#                 parameters=[], 
#                 output_mat_name='Alloy', 
#                 output_mat_properties=[])
# alloy_traveler_material_spec = setting_alloy_traveler(
#     ingredients=[alloy_material_spec],
#                 conditions=[], 
#                 parameters=[], 
#                 output_mat_name='Alloy traveler', 
#                 output_mat_properties=[]
# )
# alloy_sample_material_spec = setting_alloy_sample(
#     ingredients=[alloy_traveler_material_spec],
#                 conditions=[], 
#                 parameters=[], 
#                 output_mat_name='Alloy sample', 
#                 output_mat_properties=[]
# )



In [5]:
danial_khatamsaz_performed_source = PerformedSource('danialkh26@tamu.edu', '04/03')

### Modify iteration id below

In [6]:
iteration = "AAA"
path = "/srv/hemi01-j01/htmdec/tamu_htmdec_new/data/"

In [7]:
producible_path = Path(path)
to_display = os.path.join(producible_path, iteration)
paths = folder.Folder.make_tree(Path(to_display))
aggregate = True

In [9]:
%matplotlib inline
import json
import shutil

#TODO: complete details that can't be found without going in for loop (but should be found elsewhere, composition spaceid, composition id, etc)
# 1] Generating compositions with bayesian optimization
aggregate_summary_sheet_process_spec = ProcessSpec(
            'Aggregate summary sheet',
            template=OBJ_TEMPL['Aggregating'],
            conditions=[],
            parameters=[],
            tags=[],
            file_links=[],
            notes=None
        )
summary_sheet_material_spec = MaterialSpec(
            'Summary Sheet',
            template=OBJ_TEMPL['Summary Sheet'],
            process=aggregate_summary_sheet_process_spec,
            properties=[],
            tags=[],
            file_links=[],
            notes=None
        )
infer_compositions_process_spec = ProcessSpec(
            'Generate the next set of compositions using Bayesian Optimizations',
            template=OBJ_TEMPL['Bayesian Optimization'],
            conditions=[],
            parameters=[],
            tags=[],
            file_links=[],
            notes=None
        )
summary_sheet_ingredient_spec = IngredientSpec(
                '{} Ingredient'.format(summary_sheet_material_spec.name),
                material=summary_sheet_material_spec,
                process=infer_compositions_process_spec,
                tags = [],
                file_links=[],
                notes=None,
                labels=[],
            )
# is the compositional space fixed?
suggested_compositions_material_spec = MaterialSpec(
            'Suggested Compositions',
            template=OBJ_TEMPL['Inferred Alloy Compositions'],
            process=infer_compositions_process_spec,
            properties=[],
            tags=[],
            file_links=[],
            notes=None
        )


count = 0
for path in paths:
    if '/DED' in str(path.path) and path.depth == 5: # example: /data/AAA/DED/A/AAA01-AAA08/AAA01/C01
        continue
    if '/VAM' in str(path.path) and path.depth == 4: # example: /data/AAA/VAM/B/AAA01/T01
        vam_processing_details = json.load(open(os.path.join(path.path, 'vam-processing-details-v1.json')))
        vam_synthesis_details = json.load(open(os.path.join(path.path, 'vam-synthesis-details-v1.json')))
        vam_traveler = json.load(open(os.path.join(path.path, 'vam-traveler-v1.json')))
        full_path = str(path.path)
        composition_space_id = full_path.split('/')[1]
        composition_id = full_path.split('/')[-2]
        if count == 1:
            print(composition_id)
            # print(json.dumps(vam_processing_details, indent=4))
            # print(json.dumps(vam_synthesis_details, indent=4))
            # print(json.dumps(vam_traveler, indent=4))
        raw_jsons_dirpath = os.path.join(composition_id,'raw_jsons' )
        thin_jsons_dirpath = os.path.join(composition_id,'thin_jsons' )
        if os.path.exists(composition_id):
            shutil.rmtree(composition_id)
        os.makedirs(composition_id)
        os.makedirs(raw_jsons_dirpath)
        os.makedirs(thin_jsons_dirpath)
        mat_composition = vam_traveler['data']['Material Composition']
        batch_id = vam_traveler['data']['Sample ID']['Prod. Batch']
        yymm = vam_traveler['data']['Sample ID']['Year & Month']
        print(mat_composition['Al'])
        print(type(mat_composition['Al']))
        
        # 2] Selecting composition
        select_composition_process_spec = ProcessSpec(
            'Selecting composition for batch {} of composition {}'.format(batch_id, composition_id),
            template=OBJ_TEMPL['Selecting'],
            conditions=[],
            parameters=[],
            tags=[composition_id, yymm, batch_id],
            file_links=[],
            notes=None
        )
        
        suggested_compositions_ingredient_spec = IngredientSpec(
                '{} Ingredient'.format(suggested_compositions_material_spec.name),
                material=suggested_compositions_material_spec,
                process=select_composition_process_spec,
                tags = [],
                file_links=[],
                notes=None,
                labels=[],
            )
        
        composition_material_spec = MaterialSpec(
            'Composition {} in batch {} of composition space {}'.format(composition_id, batch_id, composition_space_id),
            template=OBJ_TEMPL['Alloy Composition'],
            process=select_composition_process_spec,
            properties=[
                    PropertyAndConditions(property=Property('Al', value=NominalReal(float(mat_composition['Al']),''), template=ATTR_TEMPL['Material Composition']), conditions = []),
                    PropertyAndConditions(property=Property('Co', value=NominalReal(float(mat_composition['Co']),''), template=ATTR_TEMPL['Material Composition']), conditions = []),
                    PropertyAndConditions(property=Property('Cr', value=NominalReal(float(mat_composition['Cr']),''), template=ATTR_TEMPL['Material Composition']), conditions = []),
                    PropertyAndConditions(property=Property('Fe', value=NominalReal(float(mat_composition['Fe']),''), template=ATTR_TEMPL['Material Composition']), conditions = []),
                    PropertyAndConditions(property=Property('Ni', value=NominalReal(float(mat_composition['Ni']),''), template=ATTR_TEMPL['Material Composition']), conditions = []),
                    PropertyAndConditions(property=Property('V', value=NominalReal(float(mat_composition['V']),''), template=ATTR_TEMPL['Material Composition']), conditions = []),
            ],
            tags=[composition_id, yymm, batch_id],
            file_links=[],
            notes=None
        )
        
        # 3] aggregate or buy materials 
        elements = []
        if aggregate:
            for p in composition_material_spec.properties:
                # print(p)
                # print(type(p))
                # print(p.name)
                # print(p.property['name'])
                # print(p['property']['value'])
            
                aggregating_material_process_spec = ProcessSpec(
                    'Aggregating {} for batch {} of composition {}'.format(p.name, batch_id, composition_id),
                    template=OBJ_TEMPL['Aggregating'],
                    conditions=[],
                    parameters=[],
                    tags=[composition_id, yymm, batch_id],
                    file_links=[],
                    notes=None
                )
                
                composition_element_material_spec = MaterialSpec(
                    '{}'.format(p.name),
                    template=OBJ_TEMPL['Composition Element'],
                    process=aggregating_material_process_spec,
                    properties=[],
                    tags=[composition_id, yymm, batch_id],
                    file_links=[],
                    notes=None
                )
                elements.append(composition_element_material_spec)
            
                IngredientSpec(
                    f'{composition_material_spec.name} Ingredient',
                    material=composition_material_spec,
                    process=aggregating_material_process_spec,
                    tags=[composition_id, yymm, batch_id],
                    file_links=[],
                    notes=None,
                    labels=[],
                )
        
        # 4] mix
        mixing_process_spec = ProcessSpec(
            'Mixing individual elements of batch {} for composition {}'.format(batch_id, composition_id),
            template=OBJ_TEMPL['Mixing'],
            conditions=[],
            parameters=[],
            tags=[composition_id, yymm, batch_id],
            file_links=[],
            notes=None
        )
        
        alloy_material_spec = MaterialSpec(
                    '{} Alloy'.format(composition_id),
                    template=OBJ_TEMPL['Alloy'],
                    process=mixing_process_spec,
                    properties=[],
                    tags=[composition_id, yymm, batch_id],
                    file_links=[],
                    notes=None
                )

        for e in elements:
            IngredientSpec(
                f'{e.name} Ingredient',
                material=e,
                process=mixing_process_spec,
                tags=[composition_id, yymm, batch_id],
                file_links=[],
                notes=None,
                labels=[],
            )
        print(composition_id)
        helpers.test(encoder, composition_id, alloy_material_spec, mode="spec")
        if count == 2:
            break
        count += 1 
        # temp = os.path.join(raw_jsons_dirpath, 'temp')
        # with open(temp, 'w') as fp:
        #     fp.write(encoder.dumps(composition_material_spec, indent=3))

        # 5]: fabricating the composition
        # a) material prep = weighting
        # b) arc melting
        # c) homogenization
        # d) Forging
        
        # 6] creating the traveler for the composition
        
        # 7] create traveler samples
        
        # 8] characterization/measurements
        
        
        
    # if path.depth == 0: 
    #     generate_composition_process_run = ProcessRun(
    #         name=" Generating composition space: {}".format(path.path.name),
    #         spec = composition_space_material_spec.process,
    #         source=danial_khatamsaz_performed_source
    #     ) 
    #     composition_space_material_run  = MaterialRun(
    #         name = "Composition space: {}".format(path.path.name),
    #         spec = composition_space_material_spec, 
    #         process = generate_composition_process_run, 
    #         sample_type='production'
            
    #     )
    # elif path.depth == 2:
    #     fabrication_method = path.path.split('/')[-2]
    #     print(path.path)
    #     print(path.path.name)
    #     print(path.depth)
    #     fabrication_process_run = ProcessRun(
    #         name="Fabricating batch {} of {} using {}".format(path.path.name, )
    #     )
        # alloy_production_batch_material_spec
    # count += 1
    # if count == 3:
    #     break

0
<class 'int'>
AAA01
process_template
Mixing
ingredient_spec
Composition AAA01 in batch B of composition space srv Ingredient
ingredient_spec
Composition AAA01 in batch B of composition space srv Ingredient
process_spec
Selecting composition for batch B of composition AAA01
process_spec
Aggregate summary sheet
ingredient_spec
Composition AAA01 in batch B of composition space srv Ingredient
material_template
Inferred Alloy Compositions
process_template
Aggregating
ingredient_spec
Ni Ingredient
material_spec
Suggested Compositions
process_spec
Mixing individual elements of batch B for composition AAA01
ingredient_spec
Summary Sheet Ingredient
process_spec
Aggregating V for batch B of composition AAA01
property_template
Material Composition
process_spec
Aggregating Cr for batch B of composition AAA01
material_template
Summary Sheet
material_spec
Ni
process_template
Bayesian Optimization
process_template
Selecting
process_spec
Aggregating Al for batch B of composition AAA01
process_spec
G

In [ ]:
# generate_composition_process_run = ProcessRun(
#             name = "Generating composition: {}".format(composition_id),
#             spec = composition_material_spec.process,
#             source = danial_khatamsaz_performed_source,
#             tags = []   
#         )
#         composition_material_run = MaterialRun(
#             name = "Composition: {}".format(composition_id),
#             process = generate_composition_process_run,
#             spec = composition_material_spec,
#             sample_type = 'experimental',
#             tags = []
#         )
#         # fab